In [1]:
import numpy as np
import pandas as pd

import os , datetime , glob

import cv2 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
%matplotlib inline

In [2]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import tensorflow as tf
import keras.preprocessing.image

In [12]:
dir_path = os.path.abspath('./data/찌개/')
data_label = list()
data_list = list()
for dirpath, dirnames, filenames  in os.walk(dir_path):
    
    data_label.append(dirnames)
    data_list.append(filenames)
#     print(dirnames )
#     print(filenames, len(filenames))
#     if filenames != len(filenames):
#         print(filenames)

In [4]:
dir_path

'C:\\Food_detecting\\data\\찌개'

In [11]:
data_label = data_label[0]

['김치찌개', '동태찌개', '된장찌개', '순두부찌개']

In [14]:
len(data_list)

5

In [18]:
data_list = data_list[1:]
data_list

[['Img_121_0000.jpg',
  'Img_121_0001.jpg',
  'Img_121_0002.jpg',
  'Img_121_0003.jpg',
  'Img_121_0004.jpg',
  'Img_121_0005.jpg',
  'Img_121_0006.jpg',
  'Img_121_0007.jpg',
  'Img_121_0008.jpg',
  'Img_121_0009.jpg',
  'Img_121_0010.jpg'],
 ['Img_121_0000.jpg',
  'Img_121_0001.jpg',
  'Img_121_0002.jpg',
  'Img_121_0003.jpg',
  'Img_121_0004.jpg',
  'Img_121_0005.jpg',
  'Img_121_0006.jpg',
  'Img_121_0007.jpg',
  'Img_121_0008.jpg',
  'Img_121_0009.jpg',
  'Img_121_0010.jpg'],
 ['Img_123_0000.jpg',
  'Img_123_0001.jpg',
  'Img_123_0002.JPG',
  'Img_123_0003.jpg',
  'Img_123_0004.JPG',
  'Img_123_0005.JPG',
  'Img_123_0006.jpg',
  'Img_123_0007.jpg',
  'Img_123_0008.JPG',
  'Img_123_0009.JPG',
  'Img_123_0010.jpg']]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode=`nearest`)

img = load_img(`data/train/cats/cat.0.jpg`)  # PIL 이미지
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=`preview`, save_prefix=`cat`, save_format=`jpeg`):
    i += 1
    if i > 20:
        break  # 이미지 20장을 생성하고 마칩니다

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


In [ ]:
model.add(Flatten())  # 이전 CNN 레이어에서 나온 3차원 배열은 1차원으로 뽑아줍니다
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
batch_size = 16

# 학습 이미지에 적용한 augmentation 인자를 지정해줍니다.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않습니다. 모델 성능을 평가할 때에는 이미지 원본을 사용합니다.
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        `data/train`,  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode=`binary`)  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        `data/validation`,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=`binary`)

test_generator = test_datagen.flow_from_directory(
        `data/validation`,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=`binary`)

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1000 // batch_size,
        validation_data=validation_generator,
        epochs=50)
model.save_weights(`first_try.h5`)  # 많은 시간을 들여 학습한 모델인 만큼, 학습 후에는 꼭 모델을 저장해줍시다.

In [ ]:
batch_size = 16

generator = datagen.flow_from_directory(
        `data/train`,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  #  라벨은 불러오지 않습니다.
        shuffle=False)  # 출력되는 병목 특징이 어디서 왔는지 알 수 있도록 입력 데이터의 순서를 유지합니다 (사전순으로 cat 1000장, dog 1000장 순서로 입력이 들어옵니다).

# 이미지를 모델에 입력시켜 결과를 가져옵니다. 본래 어떤 예측 결과가 출력되어야 하지만 모델의 일부만 가져왔기 때문에 병목 특징이 출력됩니다.
bottleneck_features_train = model.predict_generator(generator, 2000)
# 출력된 병목 데이터를 저장합니다.
np.save(open(`bottleneck_features_train.npy`, `w`), bottleneck_features_train)

generator = datagen.flow_from_directory(
        `data/validation`,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 800)
np.save(open(`bottleneck_features_validation.npy`, `w`), bottleneck_features_validation)

In [ ]:
train_data = np.load(open(`bottleneck_features_train.npy`))
# 앞서 언급한 바와 같이 병목 특징은 순서대로 추출되기 때문에 라벨 데이터는 아래와 같이 손쉽게 생성할 수 있습니다.
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open(`bottleneck_features_validation.npy`))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation=`relu`))
model.add(Dropout(0.5))
model.add(Dense(1, activation=`sigmoid`))

model.compile(optimizer=`rmsprop`,
              loss=`binary_crossentropy`,
              metrics=[`accuracy`])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(`bottleneck_fc_model.h5`)

In [ ]:
# 최상단 레이어를 생성합니다
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation=`relu`))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation=`sigmoid`))

# 기존 네트워크와 마찬가지로 이 방식에서는 최상단 레이어도 미리 학습되어 있어야 합니다.
# 앞서 학습시킨 가중치를 불러옵니다.

# VGG convolution 기반 위에 최상단 레이어를 얹습니다.
model.add(top_model)